# Geomagnetic Model Residuals through VirES

> Authors: Ashley Smith
>
> Abstract: An exploration of geomagnetic data-model residuals evaluated through VirES (along Swarm orbits).

In [ ]:
%load_ext watermark
%watermark -i -v -p viresclient,pandas,xarray,matplotlib

In [ ]:
from viresclient import SwarmRequest
import matplotlib.pyplot as plt

In [ ]:
# Use the text display of xarray instead of the html
# to avoid overly long attrs shown by default
import xarray as xr
xr.set_options(display_style="text")

## One day of residuals (data minus core field model)

In [ ]:
request = SwarmRequest()
request.set_collection("SW_OPER_MAGA_LR_1B")
request.set_products(
    measurements=["B_NEC"],
    models=["CHAOS-Core"],
    residuals=True,
    auxiliaries=["MLT", "QDLat", "SunZenithAngle"])
data = request.get_between("2019-01-01", "2019-01-02")
ds = data.as_xarray()
ds

xarray provides some convenient direct plotting tools through integration with matplotlib:

In [ ]:
ds["B_NEC_res_CHAOS-Core"].plot.line(x="Timestamp");

This allows some very compact plotting commands to create complex figures, if you know how! This kind of plotting takes a while to learn and experiment with to get right, but results in short and manageable code.

In [ ]:
facetgrid = ds.plot.scatter(x="QDLat", y="B_NEC_res_CHAOS-Core", col="NEC",
                s=0.01, hue="SunZenithAngle", cmap="hot_r", alpha=0.6)
for ax in facetgrid.axes.flat:
    ax.set_facecolor("grey")

(showing that we are in daylight more in the Southern hemisphere, while in darkness in the Northern, because the data is from January - Northern winter)

In [ ]:
facetgrid = ds.plot.scatter(x="QDLat", y="B_NEC_res_CHAOS-Core", col="NEC",
                s=0.01, hue="MLT", cmap="twilight_shifted", alpha=0.6)
for ax in facetgrid.axes.flat:
    ax.set_facecolor("grey")

(showing the rapid movement through local time sectors (MLT) when the satellite passes through polar regions)

The above figures show large residuals in the polar regions due to currents in the auroral oval. These cause greater disturbance in the Northward (N) and Eastward (E) components of the magnetic field because of the geometry of the field-aligned currents (FACs) which cause the magnetic disturbance.

## Access 1 year of data and models at 1-minute sampling

Fetch the `B_NEC` vector from Swarm Alpha and all parts of the CHAOS model (core, crust, magnetosphere) evaluated at the same points. We also access MLT, QDLat, and SunZenithAngle which are also evaluated on-the-fly.

We fetch the model values themselves as well as the measurements (instead of just the residuals as above) so that we can manipulate all the different components locally.

We apply filters to remove bad data (according to `Flags_F`) and only select geomagnetically quiet data (according to `Kp`).

Warning: Takes several minutes to process

In [ ]:
request = SwarmRequest()
request.set_collection("SW_OPER_MAGA_LR_1B")
request.set_products(
    measurements=["B_NEC"],
    models=["CHAOS-Core", "CHAOS-Static", "CHAOS-MMA-Primary", "CHAOS-MMA-Secondary"],
    auxiliaries=["MLT", "QDLat", "SunZenithAngle", "OrbitNumber"],
    sampling_step="PT60S")
request.set_range_filter("Flags_F", 0, 1)
request.set_range_filter("Kp", 0, 3)
# request.set_range_filter("SunZenithAngle", 100, 180)
data = request.get_between("2019-01-01", "2020-01-01")
ds = data.as_xarray()
ds

We can construct `xarray.DataArray`'s based on `ds`:

In [ ]:
ds["B_NEC"]

In [ ]:
ds["B_NEC"] - ds["B_NEC_CHAOS-Core"]

... and we can plot these directly:

In [ ]:
(ds["B_NEC"] - ds["B_NEC_CHAOS-Core"]).plot.line(x="Timestamp");

We can slice out a particular time window:

In [ ]:
# Selects two days
ds.sel({"Timestamp": slice("2019-01-01", "2019-01-02")})

... and subselect according to parts of the data:

In [ ]:
ds.where(ds["SunZenithAngle"] > 100, drop=True)

We can use the above to construct a plot based on part of the data

In [ ]:
# Select one day
_ds = ds.sel({"Timestamp": slice("2019-01-01", "2019-01-02")})
# Select nightside data from it
_ds_dark = _ds.where(_ds["SunZenithAngle"] > 100, drop=True)
# Append a custom residual of B-MCO-MLI-MMA
_ds_dark["B_res_full"] = _ds_dark["B_NEC"] - _ds_dark["B_NEC_CHAOS-Core"] \
                                           - _ds_dark["B_NEC_CHAOS-Static"] \
                                           - _ds_dark["B_NEC_CHAOS-MMA-Primary"] \
                                           - _ds_dark["B_NEC_CHAOS-MMA-Secondary"]
_ds_dark.plot.scatter(x="QDLat", y="B_res_full", hue="OrbitNumber", col="NEC",
                      cmap="viridis", s=0.2);

## Plotting residuals to different model combinations from one month of data

First we adjust the `Dataset` so that it contains the custom residuals themselves. We store them as both regular `data variables` and as a higher dimensional `"B_residuals"` which includes all of them. This is inefficient but is convenient for the plotting tools used below.

In [ ]:
def assign_residuals(ds):
    # Work on a copy of the Dataset so we don't disturb the original
    ds = ds.copy()
    # Assign custom residual variables
    ds["B-MCO"] = ds["B_NEC"] - ds["B_NEC_CHAOS-Core"]
    ds["B-MCO-MLI"] = ds["B-MCO"] - ds["B_NEC_CHAOS-Static"]
    ds["B-MCO-MLI-MMA"] = ds["B-MCO-MLI"] - ds["B_NEC_CHAOS-MMA-Primary"]\
                                          - ds["B_NEC_CHAOS-MMA-Secondary"]
    # Create a new DataArray to contain all these residual combinations
    da = ds["B_NEC"].copy()
    da.name = "B_residuals"
    # Expand to 3 dimensions for each residual combination to use
    da = da.expand_dims({"residuals": 3})
    da.coords["residuals"] = ["B-MCO", "B-MCO-MLI", "B-MCO-MLI-MMA"]
    # Assign the residual data to the DataArray
    da = da.copy()
    da.loc[{"residuals": "B-MCO"}] = ds["B-MCO"]
    da.loc[{"residuals": "B-MCO-MLI"}] = ds["B-MCO-MLI"]
    da.loc[{"residuals": "B-MCO-MLI-MMA"}] = ds["B-MCO-MLI-MMA"]
    # Assign the new DataArray to the original Dataset
    ds["B_residuals"] = da
    return ds

ds = assign_residuals(ds)
ds

Here we create scatter plots to show the data in different views. We select only the first month - this reduces the crowding on the figures but shows that we should seek different methods to produce summary views of the data for longer time periods. 

In [ ]:
facetgrid = (
    ds.sel({"Timestamp": slice("2019-01-01", "2019-02-01")})
      .plot.scatter(x="QDLat", y="B_residuals", col="NEC", row="residuals",
                    sharex="all", sharey="row", s=0.1,
                    hue="SunZenithAngle", cmap="hot_r"))
for ax in facetgrid.axes.flat:
    ax.set_facecolor("grey")
    ax.grid()
    ax.set_ylim((-200, 200))

In [ ]:
(
    ds
    .sel({"Timestamp": slice("2019-01-01", "2019-02-01")})
    .plot.scatter(x="Longitude", y="Latitude", hue="B-MCO-MLI-MMA",
                  s=0.1, cmap="Spectral", vmin=-50, vmax=50)
);

In [ ]:
(
    ds
    .sel({"Timestamp": slice("2019-01-01", "2019-02-01")})
    .plot.scatter(x="MLT", y="QDLat", hue="B-MCO-MLI-MMA",
                  s=0.1, cmap="Spectral", vmin=-50, vmax=50)
);

## Statistical summaries of data over the full year

In [ ]:
(
    ds
       .groupby_bins("QDLat", 90)
       .std()["B-MCO-MLI-MMA"]
       .plot.line(x="QDLat_bins")
)
plt.suptitle("Standard deviations");

to be continued